In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/renqingcuomao_datavisual

/content/drive/MyDrive/Colab Notebooks/data


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.cluster import KMeans

In [ ]:
#load all datasets
job_postings = pd.read_csv("job_postings.csv")
benefits = pd.read_csv("job_details/benefits.csv")
job_industries = pd.read_csv("job_details/job_industries.csv")
job_skills = pd.read_csv("job_details/job_skills.csv")
salaries = pd.read_csv("job_details/salaries.csv")

companies = pd.read_csv("company_details/companies.csv")
company_industries = pd.read_csv("company_details/company_industries.csv")
company_specialities = pd.read_csv("company_details/company_specialities.csv")
employee_counts = pd.read_csv("company_details/employee_counts.csv")

industries = pd.read_csv("maps/industries.csv")
skills = pd.read_csv("maps/skills.csv")

In [ ]:
#check if there is duplicate and null in job_id
duplicates = benefits['job_id'].duplicated()
num_duplicates = duplicates.sum()
num_duplicates

17678

In [ ]:
benefits['job_id'].isnull().sum()

0

In [ ]:
#Preprocess the job_details data
benefits = benefits.groupby('job_id')['type'].apply(lambda x: list(x)).reset_index()
benefits = benefits.drop('inferred', axis=1, errors='ignore')
job_industries = job_industries.merge(industries, on="industry_id", how="left")
job_industries = job_industries.groupby('job_id')['industry_name'].apply(lambda x: list(x)).reset_index()
job_skills = job_skills.merge(skills, on='skill_abr', how='left')
job_skills = job_skills.groupby('job_id')['skill_name'].apply(lambda x: list(x)).reset_index()

#Merge job_postings and job_details
merged_jobs = job_postings.merge(benefits, on="job_id", how="left")
merged_jobs = merged_jobs.merge(job_industries, on="job_id", how="left")
merged_jobs = merged_jobs.merge(job_skills, on="job_id", how="left")

merged_jobs.head(3)


,job_id,company_id,title,description,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,...,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,scraped,type,industry_name,skill_name
0,3757940104,553718.0,Hearing Care Provider,Overview\n\nHearingLife is a national hearing ...,NaN,5250.0,NaN,MONTHLY,Full-time,"Little River, SC",...,1.699090e+12,careers-demant.icims.com,0,FULL_TIME,USD,BASE_SALARY,1699138101,"[Medical insurance, Disability insurance]",[Medical Equipment Manufacturing],[Other]
1,3757940025,2192142.0,Shipping & Receiving Associate 2nd shift (Beav...,Metalcraft of Mayville\nMetalcraft of Mayville...,NaN,NaN,NaN,NaN,Full-time,"Beaver Dam, WI",...,1.699080e+12,www.click2apply.net,0,FULL_TIME,NaN,NaN,1699085420,NaN,[Industrial Machinery Manufacturing],"[Management, Manufacturing]"
2,3757938019,474443.0,"Manager, Engineering",\nThe TSUBAKI name is synonymous with excellen...,NaN,NaN,NaN,NaN,Full-time,"Bessemer, AL",...,1.699080e+12,www.click2apply.net,0,FULL_TIME,NaN,NaN,1699085644,NaN,[Automation Machinery Manufacturing],[Engineering]


In [ ]:
#Pre-process the company_details data
company_industries = company_industries.groupby('company_id')['industry'].apply(lambda x: list(x)).reset_index()
company_specialities = company_specialities.groupby('company_id')['speciality'].apply(lambda x: list(x)).reset_index()
employee_counts= employee_counts.groupby('company_id')['time_recorded'].max().reset_index()

#merge companies and company_details
merged_companies = companies.merge(company_industries, on="company_id", how="left")
merged_companies = merged_companies.merge(company_specialities, on="company_id", how="left")
merged_companies = merged_companies.merge(employee_counts, on="company_id", how="left")
merged_companies.head(3)


,company_id,name,description,company_size,state,country,city,zip_code,address,url,industry,speciality,time_recorded
0,1009,IBM,"At IBM, we do more than work. We create. We cr...",7.0,NY,US,"Armonk, New York",10504,International Business Machines Corp.,https://www.linkedin.com/company/ibm,"[Information Technology & Services, IT Service...","[Cloud, Mobile, Cognitive, Security, Research,...",1.698964e+09
1,1016,GE HealthCare,Every day millions of people feel the impact o...,7.0,0,US,Chicago,0,-,https://www.linkedin.com/company/gehealthcare,"[Hospital & Health Care, Hospitals and Health ...","[Healthcare, Biotechnology]",1.699063e+09
2,1021,GE Power,"GE Power, part of GE Vernova, is a world energ...",7.0,NY,US,Schenectady,12345,1 River Road,https://www.linkedin.com/company/gepower,"[Renewables & Environment, Renewable Energy Se...","[Distributed Power, Gasification, Generators, ...",1.699066e+09


In [ ]:
#merge job_postings and companies data
merged_post = merged_jobs.merge(merged_companies, on="company_id", how="left")
merged_post.head(5)

,job_id,company_id,title,description_x,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,...,company_size,state,country,city,zip_code,address,url,industry,speciality,time_recorded
0,3757940104,553718.0,Hearing Care Provider,Overview\n\nHearingLife is a national hearing ...,NaN,5250.00,NaN,MONTHLY,Full-time,"Little River, SC",...,5.0,New Jersey,US,Somerset,08873,580 Howard Avenue,https://www.linkedin.com/company/hearing-life,[Retail],"[Digital Hearing Instruments, behind ear, in e...",1.699131e+09
1,3757940025,2192142.0,Shipping & Receiving Associate 2nd shift (Beav...,Metalcraft of Mayville\nMetalcraft of Mayville...,NaN,NaN,NaN,NaN,Full-time,"Beaver Dam, WI",...,4.0,WI,US,Mayville,53050,1000 Metalcraft Drive,https://www.linkedin.com/company/metalcraft-of...,[Industrial Machinery Manufacturing],"[Metal Fabrication, Machining, Robotic Welding...",1.699085e+09
2,3757938019,474443.0,"Manager, Engineering",\nThe TSUBAKI name is synonymous with excellen...,NaN,NaN,NaN,NaN,Full-time,"Bessemer, AL",...,4.0,Illinois,US,Wheeling,60090,301E Marquardt Drive,https://www.linkedin.com/company/u.s.-tsubaki-...,[Automation Machinery Manufacturing],"[Roller Chain, Engineering Class Chain, Sprock...",1.699086e+09
3,3757938018,18213359.0,Cook,descriptionTitle\n\n Looking for a great oppor...,NaN,22.27,NaN,HOURLY,Full-time,"Aliso Viejo, CA",...,4.0,California,US,Altadena,91001,2212 El Molino Ave,https://www.linkedin.com/company/episcopal-com...,"[Non-profit Organization Management, Non-profi...",NaN,1.692864e+09
4,3757937095,437225.0,Principal Cloud Security Architect (Remote),"Job Summary\nAt iHerb, we are on a mission to ...",275834.0,NaN,205956.0,YEARLY,Full-time,United States,...,5.0,California,US,Irvine,92618,17400 Laguna Canyon Rd,https://www.linkedin.com/company/iherb,[Retail],"[Vitamins, Sports Nutrition, Natural Supplemen...",1.699064e+09


In [ ]:
num_rows, num_columns = merged_post.shape
num_rows, num_columns

(33246, 43)

In [ ]:
merged_post.to_csv('merged_post.csv', index=False)

In [ ]:
from google.colab import files
files.download('merged_post.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
missing_data= merged_post.isnull().sum()
sorted_missing_data = missing_data[missing_data > 0].sort_values(ascending=False)
sorted_missing_data

skills_desc                   32909
closed_time                   32074
med_salary                    31005
remote_allowed                28444
max_salary                    22135
min_salary                    22135
type                          21599
compensation_type             19894
pay_period                    19894
currency                      19894
applies                       17008
posting_domain                13558
application_url               12250
formatted_experience_level     9181
views                          7360
speciality                     7310
company_size                   2150
skill_name                     1212
description_y                   890
industry                        757
address                         718
zip_code                        717
state                           712
name                            705
city                            702
url                             701
time_recorded                   701
country                     

In [ ]:
import pandas as pd

merged_post = pd.read_csv('merged_post.csv')

print(merged_post.info())
print(merged_post.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33246 entries, 0 to 33245
Data columns (total 43 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   job_id                      33246 non-null  int64  
 1   company_id                  32592 non-null  float64
 2   title                       33246 non-null  object 
 3   description_x               33245 non-null  object 
 4   max_salary                  11111 non-null  float64
 5   med_salary                  2241 non-null   float64
 6   min_salary                  11111 non-null  float64
 7   pay_period                  13352 non-null  object 
 8   formatted_work_type         33246 non-null  object 
 9   location                    33246 non-null  object 
 10  applies                     16238 non-null  float64
 11  original_listed_time        33246 non-null  float64
 12  remote_allowed              4802 non-null   float64
 13  views                       258

In [ ]:
merged_post['max_salary'].fillna(merged_post['max_salary'].median(), inplace=True)
merged_post['med_salary'].fillna(merged_post['med_salary'].median(), inplace=True)
merged_post['min_salary'].fillna(merged_post['min_salary'].median(), inplace=True)

merged_post['pay_period'].fillna(merged_post['pay_period'].mode()[0], inplace=True)
merged_post['formatted_work_type'].fillna(merged_post['formatted_work_type'].mode()[0], inplace=True)
merged_post['remote_allowed'].fillna(False, inplace=True)
merged_post['type'].fillna(merged_post['type'].mode()[0], inplace=True)
merged_post['applies'].fillna(0, inplace=True)
merged_post['views'].fillna(0, inplace=True)

valid_times = merged_post[merged_post['closed_time'] < 1e11]
median_time = valid_times['closed_time'].median()
merged_post['closed_time'].fillna(median_time, inplace=True)
merged_post.loc[merged_post['closed_time'] > 1e11, 'closed_time'] = median_time

merged_post['country'].fillna("Unknown", inplace=True)
merged_post['state'].fillna("Unknown", inplace=True)
merged_post['city'].fillna("Unknown", inplace=True)
merged_post['zip_code'].fillna("Unknown", inplace=True)
merged_post['address'].fillna("Unknown", inplace=True)

merged_post['url'].fillna("No URL", inplace=True)
merged_post['job_posting_url'].fillna("No URL", inplace=True)
merged_post['application_url'].fillna("No URL", inplace=True)
merged_post.dropna(subset=['company_id'], inplace=True)

merged_post.to_csv('cleaned_merged_post.csv', index=False)

In [ ]:
import pandas as pd

cleaned_post = pd.read_csv('cleaned_merged_post.csv')

missing_data = cleaned_post.isnull().sum()
print("missing values stats:")
print(missing_data[missing_data > 0])

total_rows = cleaned_post.shape[0]
missing_percentage = (missing_data / total_rows) * 100
print("\npercentage:")
print(missing_percentage[missing_percentage > 0])

missing values stats:
description_x                     1
closed_time                   32592
formatted_experience_level     8622
skills_desc                   32255
posting_domain                12904
currency                      19462
compensation_type             19462
industry_name                    40
skill_name                     1023
name                             51
description_y                   236
company_size                   1496
industry                        103
speciality                     6656
time_recorded                    47
dtype: int64

percentage:
description_x                   0.003068
closed_time                   100.000000
formatted_experience_level     26.454345
skills_desc                    98.966004
posting_domain                 39.592538
currency                       59.714040
compensation_type              59.714040
industry_name                   0.122730
skill_name                      3.138807
name                            0.156480
de

In [ ]:
cleaned_post.drop(['closed_time', 'skills_desc'], axis=1, inplace=True)
cleaned_post['speciality'].fillna('Unknown', inplace=True)
median_time_recorded = cleaned_post['time_recorded'].median()
cleaned_post['time_recorded'].fillna(median_time_recorded, inplace=True)
cleaned_post.dropna(subset=['description_x'], inplace=True)

for column in ['formatted_experience_level', 'posting_domain', 'speciality', 'currency', 'compensation_type']:
    cleaned_post[column].fillna('Unknown', inplace=True)

for column in ['industry_name', 'skill_name', 'name', 'description_y', 'company_size', 'industry']:
    most_common = cleaned_post[column].mode()[0]
    cleaned_post[column].fillna(most_common, inplace=True)

missing_data_after = cleaned_post.isnull().sum()
missing_percentage_after = (missing_data_after / cleaned_post.shape[0]) * 100

print("missing values stats:")
print(missing_data_after[missing_data_after > 0])
print("\npercentage:")
print(missing_percentage_after[missing_percentage_after > 0])

cleaned_post.to_csv('further_cleaned_merged_post.csv', index=False)

missing values stats:
Series([], dtype: int64)

percentage:
Series([], dtype: float64)


In [ ]:
final_cleaned_post = pd.read_csv('further_cleaned_merged_post.csv')

num_rows, num_columns = final_cleaned_post.shape
print("rows:", num_rows)
print("columns:", num_columns)

rows: 32591
columns: 41


In [ ]:
print(list(final_cleaned_post.columns))

['job_id', 'company_id', 'title', 'description_x', 'max_salary', 'med_salary', 'min_salary', 'pay_period', 'formatted_work_type', 'location', 'applies', 'original_listed_time', 'remote_allowed', 'views', 'job_posting_url', 'application_url', 'application_type', 'expiry', 'formatted_experience_level', 'listed_time', 'posting_domain', 'sponsored', 'work_type', 'currency', 'compensation_type', 'scraped', 'type', 'industry_name', 'skill_name', 'name', 'description_y', 'company_size', 'state', 'country', 'city', 'zip_code', 'address', 'url', 'industry', 'speciality', 'time_recorded']
